In [1]:
import pandas as pd
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import re
import gensim
from gensim.utils import simple_preprocess
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import keras
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.optimizers import Adam

## 데이터 로드

In [2]:
# # 집
# df=pd.read_csv('C:/Users/yeons/OneDrive/바탕 화면/tb_cargomylist_need_columns.csv', encoding='cp949')
# df

In [58]:
# 학교
df=pd.read_csv('C:/Users/user/OneDrive - kyonggi.ac.kr/바탕 화면/화물맨프로젝트/화물맨 프로젝트/Ipynb/오토인코더추천시스템/tb_cargomylist_need_columns.csv', encoding='cp949')
df

C:\Users\user\AppData\Local\Temp\ipykernel_18896\1352208895.py:2: DtypeWarning: Columns (27,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('C:/Users/user/OneDrive - kyonggi.ac.kr/바탕 화면/화물맨프로젝트/화물맨 프로젝트/Ipynb/오토인코더추천시스템/tb_cargomylist_need_columns.csv', encoding='cp949')


,Unnamed: 0,intSeq,intStatus,bShare,intOwMemberSeq,strUpZone,strUpAreaCode,strUpJAddress,strDwZone,strDwAreaCode,...,strDownloadType,intDistance,intDefaultPrice,fltRealWeight,intActApp,intUpDateType,intDwDateType,intCarManageSeq,intMemberSeq,intFeeSend
0,0,92218976,3,1,57037.0,41.0,H050004,경기 광주 도척면,30.0,F010005,...,일반,116,0,7.0,999,1.0,1.0,11017.0,NaN,NaN
1,1,92218977,3,1,57037.0,41.0,H050004,경기 광주 도척면,43.0,J100000,...,일반,113,0,7.0,999,1.0,1.0,11017.0,NaN,NaN
2,2,92218978,3,1,63816.0,41.0,H080017,경기 김포 학운리,41.0,H230304,...,일반,100,0,26.0,2,1.0,1.0,11017.0,12953.0,NaN
3,3,92218979,6,1,55238.0,28.0,D000000,인천,26.0,B000000,...,일반,408,0,27.5,202,1.0,2.0,11017.0,NaN,NaN
4,4,92218980,3,1,41258.0,41.0,H300000,경기 하남,41.0,H060000,...,일반,14,135000,10.0,4,1.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885028,384471,1221530,6,1,38193.0,41.0,H260008,경기 이천 마장면,41.0,H120300,...,지게차,48,0,1.0,93,2.0,2.0,\N,0.0,NaN
1885029,384472,1221531,6,1,38193.0,41.0,H260008,경기 이천 마장면,41.0,H120300,...,지게차,48,0,1.0,93,2.0,2.0,\N,0.0,NaN
1885030,384473,1221532,6,1,38193.0,41.0,H260008,경기 이천 마장면,41.0,H120300,...,지게차,48,0,1.0,93,2.0,2.0,\N,0.0,NaN
1885031,384474,1221533,6,1,53524.0,30.0,F020000,대전 동구,46.0,K080000,...,지게차,231,100000,5.0,2,1.0,2.0,\N,0.0,NaN


In [59]:
# "intOwMemberSeq" 열의 값을 자연수 형태로 변환
df['intOwMemberSeq'] = df['intOwMemberSeq'].fillna(0)
df['intOwMemberSeq'] = df['intOwMemberSeq'].astype(int)

print(df)

         Unnamed: 0    intSeq  intStatus  bShare  intOwMemberSeq  strUpZone  \
0                 0  92218976          3       1           57037       41.0   
1                 1  92218977          3       1           57037       41.0   
2                 2  92218978          3       1           63816       41.0   
3                 3  92218979          6       1           55238       28.0   
4                 4  92218980          3       1           41258       41.0   
...             ...       ...        ...     ...             ...        ...   
1885028      384471   1221530          6       1           38193       41.0   
1885029      384472   1221531          6       1           38193       41.0   
1885030      384473   1221532          6       1           38193       41.0   
1885031      384474   1221533          6       1           53524       30.0   
1885032      384475   1221534          6       1           73964       41.0   

        strUpAreaCode strUpJAddress  strDwZone strD

In [60]:
# 열 삭제 
df = df.drop('Unnamed: 0',axis=1)

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1885033 entries, 0 to 1885032
Data columns (total 32 columns):
 #   Column           Dtype  
---  ------           -----  
 0   intSeq           int64  
 1   intStatus        int64  
 2   bShare           int64  
 3   intOwMemberSeq   int32  
 4   strUpZone        float64
 5   strUpAreaCode    object 
 6   strUpJAddress    object 
 7   strDwZone        float64
 8   strDwAreaCode    object 
 9   strDwJAddress    object 
 10  strCarType       object 
 11  intCarTypecode   float64
 12  strCarSize       object 
 13  intCarSizecode   float64
 14  intCarUpType     int64  
 15  strPaytype       object 
 16  intPaytypecode   int64  
 17  intTotalPrice    int64  
 18  intMemberPrice   int64  
 19  intArrangePrice  int64  
 20  strGoodsType     object 
 21  strUploadType    object 
 22  strDownloadType  object 
 23  intDistance      int64  
 24  intDefaultPrice  int64  
 25  fltRealWeight    float64
 26  intActApp        object 
 27  intUpDateTyp

## 오토인코더를 사용하기 위해 object 타입 수치형 데이터로 변환

### - labelencoder 사용

In [62]:
encoder = LabelEncoder()
df['strUpAreaCode'] = encoder.fit_transform(df['strUpAreaCode'].astype(str))
df['strUpJAddress'] = encoder.fit_transform(df['strUpJAddress'].astype(str))
df['strDwJAddress'] = encoder.fit_transform(df['strDwJAddress'].astype(str))
df['strUpAreaCode'] = encoder.fit_transform(df['strUpAreaCode'].astype(str))
df['strDwAreaCode'] = encoder.fit_transform(df['strDwAreaCode'].astype(str))
df['strDwJAddress'] = encoder.fit_transform(df['strDwJAddress'].astype(str))
df['strCarType'] = encoder.fit_transform(df['strCarType'].astype(str))
df['strCarSize'] = encoder.fit_transform(df['strCarSize'].astype(str))
df['strPaytype'] = encoder.fit_transform(df['strPaytype'].astype(str))
df['strGoodsType'] = encoder.fit_transform(df['strGoodsType'].astype(str))
df['strUploadType'] = encoder.fit_transform(df['strUploadType'].astype(str))
df['strDownloadType'] = encoder.fit_transform(df['strDownloadType'].astype(str))
df['intActApp'] = encoder.fit_transform(df['intActApp'].astype(str))
df['intCarManageSeq'] = encoder.fit_transform(df['intCarManageSeq'].astype(str))

In [63]:
df

,intSeq,intStatus,bShare,intOwMemberSeq,strUpZone,strUpAreaCode,strUpJAddress,strDwZone,strDwAreaCode,strDwJAddress,...,strDownloadType,intDistance,intDefaultPrice,fltRealWeight,intActApp,intUpDateType,intDwDateType,intCarManageSeq,intMemberSeq,intFeeSend
0,92218976,3,1,57037,41.0,276,2209,30.0,1083,29987,...,1,116,0,7.0,12,1.0,1.0,4,NaN,NaN
1,92218977,3,1,57037,41.0,276,2210,43.0,2660,61100,...,1,113,0,7.0,12,1.0,1.0,4,NaN,NaN
2,92218978,3,1,63816,41.0,339,3384,41.0,1947,3505,...,1,100,0,26.0,2,1.0,1.0,4,12953.0,NaN
3,92218979,6,1,55238,28.0,3995,28850,26.0,379,31401,...,1,408,0,27.5,3,1.0,2.0,4,NaN,NaN
4,92218980,3,1,41258,41.0,1008,11509,41.0,1459,48169,...,1,14,135000,10.0,8,1.0,1.0,9,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885028,1221530,6,1,38193,41.0,874,8495,41.0,1612,64693,...,2,48,0,1.0,10,2.0,2.0,8,0.0,NaN
1885029,1221531,6,1,38193,41.0,874,8495,41.0,1612,64693,...,2,48,0,1.0,10,2.0,2.0,8,0.0,NaN
1885030,1221532,6,1,38193,41.0,874,8495,41.0,1612,64693,...,2,48,0,1.0,10,2.0,2.0,8,0.0,NaN
1885031,1221533,6,1,53524,30.0,4351,23374,46.0,2949,45702,...,2,231,100000,5.0,2,1.0,2.0,8,0.0,NaN


## 오토인코더를 사용하기 위해 입력 데이터를 정규화

In [64]:
# 데이터를 정규화합니다.
scaler = StandardScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df),columns=df.columns)
df_normalized

,intSeq,intStatus,bShare,intOwMemberSeq,strUpZone,strUpAreaCode,strUpJAddress,strDwZone,strDwAreaCode,strDwJAddress,...,strDownloadType,intDistance,intDefaultPrice,fltRealWeight,intActApp,intUpDateType,intDwDateType,intCarManageSeq,intMemberSeq,intFeeSend
0,0.285695,-0.990183,0.034152,0.817190,0.123499,-1.329024,-1.572152,-1.011072,-0.918296,-0.170030,...,-0.328421,0.004006,-0.684241,-1.075246,1.663247,-0.389139,-0.908531,-0.784032,NaN,NaN
1,0.285695,-0.990183,0.034152,0.817190,0.123499,-1.329024,-1.572075,0.448039,0.307204,1.392233,...,-0.328421,0.004006,-0.684241,-1.075246,1.663247,-0.389139,-0.908531,-0.784032,NaN,NaN
2,0.285695,-0.990183,0.034152,1.146550,0.123499,-1.277805,-1.481069,0.223560,-0.246875,-1.499758,...,-0.328421,0.004005,-0.684241,1.178400,-1.758640,-0.389139,-0.908531,-0.784032,-0.307990,NaN
3,0.285695,0.966049,0.034152,0.729785,-1.623514,1.694531,0.493004,-1.460029,-1.465381,-0.099029,...,-0.328421,0.004034,-0.684241,1.356320,-1.416451,-0.389139,0.300922,-0.784032,NaN,NaN
4,0.285695,-0.990183,0.034152,0.050563,0.123499,-0.733906,-0.851235,0.223560,-0.626104,0.742935,...,-0.328421,0.003997,0.147795,-0.719407,0.294492,-0.389139,-0.908531,1.168506,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885028,-1.934176,0.966049,0.034152,-0.098350,0.123499,-0.842849,-1.084874,0.223560,-0.507206,1.572647,...,1.613138,0.004000,-0.684241,-1.786924,0.978870,1.647601,0.300922,0.777998,-0.807495,NaN
1885029,-1.934176,0.966049,0.034152,-0.098350,0.123499,-0.842849,-1.084874,0.223560,-0.507206,1.572647,...,1.613138,0.004000,-0.684241,-1.786924,0.978870,1.647601,0.300922,0.777998,-0.807495,NaN
1885030,-1.934176,0.966049,0.034152,-0.098350,0.123499,-0.842849,-1.084874,0.223560,-0.507206,1.572647,...,1.613138,0.004000,-0.684241,-1.786924,0.978870,1.647601,0.300922,0.777998,-0.807495,NaN
1885031,-1.934176,0.966049,0.034152,0.646510,-1.354743,1.983959,0.068515,0.784757,0.531788,0.619060,...,1.613138,0.004017,-0.067918,-1.312472,-1.758640,-0.389139,0.300922,0.777998,-0.807495,NaN


In [65]:
# 1000개만
df1 = df_normalized.iloc[:100000, :]
df1

,intSeq,intStatus,bShare,intOwMemberSeq,strUpZone,strUpAreaCode,strUpJAddress,strDwZone,strDwAreaCode,strDwJAddress,...,strDownloadType,intDistance,intDefaultPrice,fltRealWeight,intActApp,intUpDateType,intDwDateType,intCarManageSeq,intMemberSeq,intFeeSend
0,0.285695,-0.990183,0.034152,0.817190,0.123499,-1.329024,-1.572152,-1.011072,-0.918296,-0.170030,...,-0.328421,0.004006,-0.684241,-1.075246,1.663247,-0.389139,-0.908531,-0.784032,NaN,NaN
1,0.285695,-0.990183,0.034152,0.817190,0.123499,-1.329024,-1.572075,0.448039,0.307204,1.392233,...,-0.328421,0.004006,-0.684241,-1.075246,1.663247,-0.389139,-0.908531,-0.784032,NaN,NaN
2,0.285695,-0.990183,0.034152,1.146550,0.123499,-1.277805,-1.481069,0.223560,-0.246875,-1.499758,...,-0.328421,0.004005,-0.684241,1.178400,-1.758640,-0.389139,-0.908531,-0.784032,-0.30799,NaN
3,0.285695,0.966049,0.034152,0.729785,-1.623514,1.694531,0.493004,-1.460029,-1.465381,-0.099029,...,-0.328421,0.004034,-0.684241,1.356320,-1.416451,-0.389139,0.300922,-0.784032,NaN,NaN
4,0.285695,-0.990183,0.034152,0.050563,0.123499,-0.733906,-0.851235,0.223560,-0.626104,0.742935,...,-0.328421,0.003997,0.147795,-0.719407,0.294492,-0.389139,-0.908531,1.168506,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.288135,0.966049,0.034152,1.198196,0.123499,-1.259106,-1.452697,0.223560,-0.133417,-1.309754,...,-0.328421,0.004006,1.072279,0.443000,0.294492,-0.389139,2.719829,1.168506,NaN,NaN
99996,0.288135,0.966049,0.034152,1.198196,0.123499,-1.259106,-1.452697,0.223560,-0.133417,-1.309754,...,-0.328421,0.004006,1.072279,0.443000,0.294492,-0.389139,-0.908531,1.168506,NaN,NaN
99997,0.288135,0.966049,0.034152,0.015582,0.123499,-1.308699,-1.544788,-3.143619,-1.725712,0.141037,...,-0.328421,0.003999,-0.684241,1.297013,-2.443017,-0.389139,2.719829,-0.784032,NaN,NaN
99998,0.288135,0.966049,0.034152,0.729785,-1.623514,1.694531,0.493004,-1.460029,-1.451393,-0.064784,...,-0.328421,0.004034,-0.684241,1.356320,-1.416451,5.721081,2.719829,-0.784032,NaN,NaN


In [66]:
df1['intOwMemberSeq'] = df['intOwMemberSeq']
df1

C:\Users\user\AppData\Local\Temp\ipykernel_18896\3184527928.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['intOwMemberSeq'] = df['intOwMemberSeq']


,intSeq,intStatus,bShare,intOwMemberSeq,strUpZone,strUpAreaCode,strUpJAddress,strDwZone,strDwAreaCode,strDwJAddress,...,strDownloadType,intDistance,intDefaultPrice,fltRealWeight,intActApp,intUpDateType,intDwDateType,intCarManageSeq,intMemberSeq,intFeeSend
0,0.285695,-0.990183,0.034152,57037,0.123499,-1.329024,-1.572152,-1.011072,-0.918296,-0.170030,...,-0.328421,0.004006,-0.684241,-1.075246,1.663247,-0.389139,-0.908531,-0.784032,NaN,NaN
1,0.285695,-0.990183,0.034152,57037,0.123499,-1.329024,-1.572075,0.448039,0.307204,1.392233,...,-0.328421,0.004006,-0.684241,-1.075246,1.663247,-0.389139,-0.908531,-0.784032,NaN,NaN
2,0.285695,-0.990183,0.034152,63816,0.123499,-1.277805,-1.481069,0.223560,-0.246875,-1.499758,...,-0.328421,0.004005,-0.684241,1.178400,-1.758640,-0.389139,-0.908531,-0.784032,-0.30799,NaN
3,0.285695,0.966049,0.034152,55238,-1.623514,1.694531,0.493004,-1.460029,-1.465381,-0.099029,...,-0.328421,0.004034,-0.684241,1.356320,-1.416451,-0.389139,0.300922,-0.784032,NaN,NaN
4,0.285695,-0.990183,0.034152,41258,0.123499,-0.733906,-0.851235,0.223560,-0.626104,0.742935,...,-0.328421,0.003997,0.147795,-0.719407,0.294492,-0.389139,-0.908531,1.168506,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.288135,0.966049,0.034152,64879,0.123499,-1.259106,-1.452697,0.223560,-0.133417,-1.309754,...,-0.328421,0.004006,1.072279,0.443000,0.294492,-0.389139,2.719829,1.168506,NaN,NaN
99996,0.288135,0.966049,0.034152,64879,0.123499,-1.259106,-1.452697,0.223560,-0.133417,-1.309754,...,-0.328421,0.004006,1.072279,0.443000,0.294492,-0.389139,-0.908531,1.168506,NaN,NaN
99997,0.288135,0.966049,0.034152,40538,0.123499,-1.308699,-1.544788,-3.143619,-1.725712,0.141037,...,-0.328421,0.003999,-0.684241,1.297013,-2.443017,-0.389139,2.719829,-0.784032,NaN,NaN
99998,0.288135,0.966049,0.034152,55238,-1.623514,1.694531,0.493004,-1.460029,-1.451393,-0.064784,...,-0.328421,0.004034,-0.684241,1.356320,-1.416451,5.721081,2.719829,-0.784032,NaN,NaN


In [67]:
df1['intSeq'] = df['intSeq']
df1

C:\Users\user\AppData\Local\Temp\ipykernel_18896\997612670.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['intSeq'] = df['intSeq']


,intSeq,intStatus,bShare,intOwMemberSeq,strUpZone,strUpAreaCode,strUpJAddress,strDwZone,strDwAreaCode,strDwJAddress,...,strDownloadType,intDistance,intDefaultPrice,fltRealWeight,intActApp,intUpDateType,intDwDateType,intCarManageSeq,intMemberSeq,intFeeSend
0,92218976,-0.990183,0.034152,57037,0.123499,-1.329024,-1.572152,-1.011072,-0.918296,-0.170030,...,-0.328421,0.004006,-0.684241,-1.075246,1.663247,-0.389139,-0.908531,-0.784032,NaN,NaN
1,92218977,-0.990183,0.034152,57037,0.123499,-1.329024,-1.572075,0.448039,0.307204,1.392233,...,-0.328421,0.004006,-0.684241,-1.075246,1.663247,-0.389139,-0.908531,-0.784032,NaN,NaN
2,92218978,-0.990183,0.034152,63816,0.123499,-1.277805,-1.481069,0.223560,-0.246875,-1.499758,...,-0.328421,0.004005,-0.684241,1.178400,-1.758640,-0.389139,-0.908531,-0.784032,-0.30799,NaN
3,92218979,0.966049,0.034152,55238,-1.623514,1.694531,0.493004,-1.460029,-1.465381,-0.099029,...,-0.328421,0.004034,-0.684241,1.356320,-1.416451,-0.389139,0.300922,-0.784032,NaN,NaN
4,92218980,-0.990183,0.034152,41258,0.123499,-0.733906,-0.851235,0.223560,-0.626104,0.742935,...,-0.328421,0.003997,0.147795,-0.719407,0.294492,-0.389139,-0.908531,1.168506,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,92318973,0.966049,0.034152,64879,0.123499,-1.259106,-1.452697,0.223560,-0.133417,-1.309754,...,-0.328421,0.004006,1.072279,0.443000,0.294492,-0.389139,2.719829,1.168506,NaN,NaN
99996,92318974,0.966049,0.034152,64879,0.123499,-1.259106,-1.452697,0.223560,-0.133417,-1.309754,...,-0.328421,0.004006,1.072279,0.443000,0.294492,-0.389139,-0.908531,1.168506,NaN,NaN
99997,92318975,0.966049,0.034152,40538,0.123499,-1.308699,-1.544788,-3.143619,-1.725712,0.141037,...,-0.328421,0.003999,-0.684241,1.297013,-2.443017,-0.389139,2.719829,-0.784032,NaN,NaN
99998,92318976,0.966049,0.034152,55238,-1.623514,1.694531,0.493004,-1.460029,-1.451393,-0.064784,...,-0.328421,0.004034,-0.684241,1.356320,-1.416451,5.721081,2.719829,-0.784032,NaN,NaN


## 유저 만들기 및 특성, 가중치 부여

In [68]:
# 데이터 프레임에서 유저 ID 추출하기
user_id_array = df1['intOwMemberSeq'].unique()
user_id_array

array([57037, 63816, 55238, ..., 48300, 25913, 71663])

In [73]:
# 유저 특성 추출하기
# 차량종류, 차량톤수 ,결제 방법, 기사운임,상차종류,하차종류
user_features1 = df1.set_index('intOwMemberSeq')[['strPaytype','strUploadType','strCarType','intDistance','strUpAreaCode','intMemberPrice','strCarSize']].groupby('intOwMemberSeq')

In [74]:
# 결측값을 0으로 대체
user_features = user_features1.fillna(0)

print(user_features)

                strPaytype  strUploadType  strCarType  intDistance  \
intOwMemberSeq                                                       
57037            -0.169079       -0.33145   -1.544589     0.004006   
57037            -0.169079       -0.33145   -1.544589     0.004006   
57037            -0.169079       -0.33145   -1.544589     0.003995   
57037            -0.169079       -0.33145   -1.544589     0.003995   
57037            -0.169079       -0.33145   -1.544589     0.003995   
...                    ...            ...         ...          ...   
72014            -2.556742       -0.33145    0.483332     0.003998   
33756            -0.169079       -0.33145    0.483332     0.004007   
48300            -0.169079       -0.33145    0.483332     0.004000   
25913             3.014471       -0.33145    0.483332     0.004029   
71663            -0.169079       -0.33145    0.483332     0.004009   

                strUpAreaCode  intMemberPrice  strCarSize  
intOwMemberSeq               

In [75]:
# 각 특성의 가중치 설정
strPaytype_weight = -0.5
strUploadType_weight  = -0.3
strCarType_weight  = 1.51
intMemberPrice_weight = -0.51
strUpAreaCode_weight = 1.25
strCarSize_weight = 0.7
intDistance_weight = 0.7

In [76]:
# 각 특성에 가중치 부여
user_features['strPaytype'] *= strPaytype_weight 
user_features['strUploadType'] *= strUploadType_weight 
user_features['strCarType'] *= strCarType_weight 
user_features['intMemberPrice'] *= intMemberPrice_weight
user_features['strUpAreaCode'] *= strUpAreaCode_weight
item_features['intDistance'] *= intDistance_weight
user_features['strCarSize'] *= strCarSize_weight

In [77]:
# 유저 목록 출력
print(user_features.head())

                strPaytype  strUploadType  strCarType  intDistance  \
intOwMemberSeq                                                       
57037              0.08454       0.099435   -2.332329     0.004006   
57037              0.08454       0.099435   -2.332329     0.004006   
57037              0.08454       0.099435   -2.332329     0.003995   
57037              0.08454       0.099435   -2.332329     0.003995   
57037              0.08454       0.099435   -2.332329     0.003995   

                strUpAreaCode  intMemberPrice  strCarSize  
intOwMemberSeq                                             
57037               -1.661280        0.109020   -0.851758  
57037               -1.661280        0.141461   -0.851758  
57037               -1.114536        0.173901   -0.851758  
57037               -1.114536        0.173901   -0.851758  
57037               -1.114536        0.141461   -0.851758  


## 아이템 만들기 및 특성, 가중치 부여

In [78]:
# 데이터 프레임에서 아이템 ID 추출하기
item_id = df['intSeq'].unique()
item_id

array([92218976, 92218977, 92218978, ...,  1221532,  1221533,  1221534],
      dtype=int64)

In [79]:
# 아이템 특성 추출하기
# 상차지역, 상차위치, 상차지번,하차지역, 하차위치, 하자지번, 화물 형태, 실거리, 화물 실중량 
#item_features = df1.groupby('intSeq')[['strUpZone','strUpAreaCode','strUpJAddress','strDwZone','strDwAreaCode','strDwJAddress','strGoodsType','intDistance','fltRealWeight']]

item_features1 = df1.groupby('intSeq')[['strUpJAddress','strUploadType','strDownloadType','strDwJAddress','strGoodsType','intDistance','fltRealWeight']]


In [80]:
# 결측값을 0으로 대체
item_features = item_features1.fillna(0)

# 아이템 목록 출력
print(item_features.head())

   strUpJAddress  strUploadType  strDownloadType  strDwJAddress  strGoodsType  \
0      -1.572152       -0.33145        -0.328421      -0.170030      0.485106   
1      -1.572075       -0.33145        -0.328421       1.392233      0.485106   
2      -1.481069       -0.33145        -0.328421      -1.499758      0.485106   
3       0.493004       -0.33145        -0.328421      -0.099029      0.485106   
4      -0.851235       -0.33145        -0.328421       0.742935      0.485106   

   intDistance  fltRealWeight  
0     0.004006      -1.075246  
1     0.004006      -1.075246  
2     0.004005       1.178400  
3     0.004034       1.356320  
4     0.003997      -0.719407  


In [81]:
# 각 특성의 가중치 설정
strUpJAddress_weight = - 2.0
strUploadType_weight = 1.3
strDownloadType_weight = 0.4
strDwJAddress_weight = 0.9
strGoodsType_weight = - 1.8
intDistance_weight = 0.7
fltRealWeight_weight = - 1.5 

In [82]:
# 각 특성에 가중치 부여
item_features['strUpJAddress'] *= strUpJAddress_weight 
item_features['strUploadType'] *= strUploadType_weight 
item_features['strDownloadType'] *= strDownloadType_weight
item_features['strDwJAddress'] *= strDwJAddress_weight
item_features['strGoodsType'] *= strGoodsType_weight
item_features['intDistance'] *= intDistance_weight
item_features['fltRealWeight'] *= fltRealWeight_weight

In [83]:
# 아이템 목록 출력
print(item_features.head())

   strUpJAddress  strUploadType  strDownloadType  strDwJAddress  strGoodsType  \
0       3.144305      -0.430885        -0.131368      -0.153027     -0.873191   
1       3.144150      -0.430885        -0.131368       1.253010     -0.873191   
2       2.962137      -0.430885        -0.131368      -1.349783     -0.873191   
3      -0.986008      -0.430885        -0.131368      -0.089126     -0.873191   
4       1.702470      -0.430885        -0.131368       0.668641     -0.873191   

   intDistance  fltRealWeight  
0     0.002804       1.612869  
1     0.002804       1.612869  
2     0.002803      -1.767600  
3     0.002824      -2.034479  
4     0.002798       1.079111  


In [84]:
user_features.shape

(100000, 7)

In [85]:
item_features.shape

(100000, 7)

# 추천시스템 구축

In [92]:
class AutoencoderRecommender:
    def __init__(self, user_features, item_features, latent_dim=14, encoding_dim=14):
        self.user_features = user_features
        self.item_features = item_features
        self.encoding_dim = encoding_dim
        self.scaler = MinMaxScaler()
        self.user_encoder, self.user_autoencoder = self.build_autoencoder(self.user_features.shape[1])
        self.item_encoder, self.item_autoencoder = self.build_autoencoder(self.item_features.shape[1])

    def build_autoencoder(self, input_dim):
        # Encoder
        input_layer = Input(shape=(input_dim,))
        encoded = Dense(50, activation='ReLU')(input_layer)
        encoded = Dense(15, activation='ReLU')(encoded)
        encoded = Dense(self.encoding_dim, activation='sigmoid')(encoded)

        # Decoder
        decoded = Dense(15, activation='ReLU')(encoded)
        decoded = Dense(50, activation='ReLU')(decoded)
        decoded = Dense(input_dim, activation='sigmoid')(decoded)

        # Autoencoder
        autoencoder = Model(inputs=input_layer, outputs=decoded)
        autoencoder.compile(optimizer='adam', loss='mse')

        # Encoder
        encoder = Model(inputs=input_layer, outputs=encoded)

        return encoder, autoencoder

    def fit(self, user_epochs=10, item_epochs=10, batch_size=32):
        user_features = self.user_features.values
        item_features = self.item_features.values
        #user_features = self.scaler.fit_transform(user_features)
        #item_features = self.scaler.fit_transform(item_features)

        self.user_autoencoder.fit(user_features, user_features,
                                  epochs=user_epochs,
                                  batch_size=batch_size,
                                  shuffle=True)
        self.item_autoencoder.fit(item_features, item_features,
                                  epochs=item_epochs,
                                  batch_size=batch_size,
                                  shuffle=True)

        # Encode user features and item features
        self.user_features_encoded = self.user_encoder.predict(user_features)
        self.item_features_encoded = self.item_encoder.predict(item_features)
    def recommend_items(self, user_id, n_items=10):
        # 사용자 특성
        user_features = self.user_features.loc[user_id].values

        # 특정 유저 인코딩 벡터
        user_encoded = self.user_encoder.predict([user_features])

        # 전체 유저 인코딩 벡터
        user_features_encoded = self.user_encoder.predict(self.user_features.values)

        # 특정 아이템 인코딩 벡터
        item_encoded = self.item_encoder.predict(self.item_features.values)

        # 전체 아이템 인코딩 벡터
        item_features_encoded = self.item_encoder.predict(self.item_features.values)

        # 유저간 유사도 계산
        user_similarities = cosine_similarity(user_encoded, user_features_encoded)

        # 가장 유사한 유저들 선정 - 같은 유저는 제외
        similar_users = np.argsort(-user_similarities[0])[1:]

        # 유사한 유저들의 아이템 벡터
        similar_users_item_features = self.item_encoder.predict(self.user_features.values[similar_users])

        # 아이템 간의 유사도 계산하기
        item_similarities = cosine_similarity(item_encoded, similar_users_item_features)

        # 유사한 아이템 선정하기
        item_indices = np.argsort(-item_similarities[0])[:n_items]

        # 평점 시스템 계산하기
        predicted_ratings = np.dot(item_similarities[0][item_indices], item_features_encoded[item_indices])

        # 최저점 최대점 설정하기
        min_rating = 0
        max_rating = 5
        predicted_ratings = (predicted_ratings - np.min(predicted_ratings)) / (
                np.max(predicted_ratings) - np.min(predicted_ratings))
        predicted_ratings = predicted_ratings * (max_rating - min_rating) + min_rating

        # 추천 아이템 선정하기
        recommended_items = self.item_features.index[item_indices].values.tolist()

        # 유사도와 추천 아이템을 함께 반환
        item_similarities = item_similarities[0][item_indices].tolist()
        recommended_items_with_similarities = list(zip(recommended_items, item_similarities))

        return recommended_items_with_similarities, predicted_ratings, user_similarities, item_similarities
    
    
    def get_item_features(self, item_id):
        # 특정 아이템 인코딩 벡터
        item_encoded = self.item_encoder.predict(self.item_features.values)
        
        # 특정 아이템의 인덱스 찾기
        item_index = self.item_features.index.get_loc(item_id)
        
        # 특정 아이템의 특징 디코딩
        decoded_features = self.item_autoencoder.predict(np.expand_dims(item_encoded[item_index], axis=0))

        return decoded_features

# 오토인코더 학습

In [89]:
# AutoencoderRecommender 객체를 생성한다.
recommender = AutoencoderRecommender(user_features, item_features, latent_dim=14, encoding_dim=14)

# 오토인코더 모델을 학습한다.
recommender.fit(user_epochs=100, item_epochs=100, batch_size=64)

Epoch 1/100
1563/1563 [==============================] - 1s 503us/step - loss: 0.6195
Epoch 2/100
1563/1563 [==============================] - 1s 470us/step - loss: 0.5813
Epoch 3/100
1563/1563 [==============================] - 1s 468us/step - loss: 0.5804
Epoch 4/100
1563/1563 [==============================] - 1s 476us/step - loss: 0.5794
Epoch 5/100
1563/1563 [==============================] - 1s 472us/step - loss: 0.5788
Epoch 6/100
1563/1563 [==============================] - 1s 474us/step - loss: 0.5786
Epoch 7/100
1563/1563 [==============================] - 1s 471us/step - loss: 0.5785
Epoch 8/100
1563/1563 [==============================] - 1s 461us/step - loss: 0.5784
Epoch 9/100
1563/1563 [==============================] - 1s 457us/step - loss: 0.5784
Epoch 10/100
1563/1563 [==============================] - 1s 455us/step - loss: 0.5784
Epoch 11/100
1563/1563 [==============================] - 1s 454us/step - loss: 0.5783
Epoch 12/100
1563/1563 [============================

# 유저 입력

In [90]:
user_id = input("Enter the user ID: ")
user_id = float(user_id)
n_items = 100

recommendations, predicted_ratings, user_similarities, item_similarities = recommender.recommend_items(user_id, n_items)

print(f"{user_id} 사용자에게 추천하는 아이템:")
for i, ((item, similarity), rating, item_similarity) in enumerate(zip(recommendations, predicted_ratings, item_similarities)):
    random_similarity = np.random.choice(user_similarities[:, i])
    similairy = np.mean(user_similarities[:, i])
    print(f"- 추천 아이템: {item}, 아이템 유사도: {item_similarity:.4f}, 평점: {rating:.4f},유저 유사도:{similairy:.4f}")
    #random_similarity

Enter the user ID: 55238
3125/3125 [==============================] - 1s 301us/step
55238.0 사용자에게 추천하는 아이템:
- 추천 아이템: 37686, 아이템 유사도: 0.9642, 평점: 4.3357,유저 유사도:0.7956
- 추천 아이템: 13680, 아이템 유사도: 0.9640, 평점: 3.0541,유저 유사도:0.7944
- 추천 아이템: 13712, 아이템 유사도: 0.9639, 평점: 3.0421,유저 유사도:0.7902
- 추천 아이템: 37543, 아이템 유사도: 0.9639, 평점: 4.0451,유저 유사도:0.7902
- 추천 아이템: 29748, 아이템 유사도: 0.9638, 평점: 3.4219,유저 유사도:0.7915
- 추천 아이템: 37255, 아이템 유사도: 0.9635, 평점: 4.0923,유저 유사도:0.7928
- 추천 아이템: 36785, 아이템 유사도: 0.9627, 평점: 0.0000,유저 유사도:0.7908
- 추천 아이템: 32161, 아이템 유사도: 0.9626, 평점: 5.0000,유저 유사도:0.7956
- 추천 아이템: 32160, 아이템 유사도: 0.9626, 평점: 1.6228,유저 유사도:0.7999
- 추천 아이템: 14406, 아이템 유사도: 0.9611, 평점: 4.9988,유저 유사도:0.7915
- 추천 아이템: 14405, 아이템 유사도: 0.9611, 평점: 4.3015,유저 유사도:0.7999
- 추천 아이템: 14407, 아이템 유사도: 0.9611, 평점: 4.7645,유저 유사도:0.7917
- 추천 아이템: 34551, 아이템 유사도: 0.9607, 평점: 0.6080,유저 유사도:0.7917
- 추천 아이템: 31291, 아이템 유사도: 0.9606, 평점: 4.1471,유저 유사도:0.7927


In [161]:
df2=pd.read_csv('C:/Users/user/OneDrive - kyonggi.ac.kr/바탕 화면/화물맨프로젝트/화물맨 프로젝트/Ipynb/오토인코더추천시스템/tb_cargomylist_need_columns.csv', encoding='cp949')
selected_columns = df2[['strUpJAddress', 'strUploadType', 'strDownloadType','strDwJAddress','strGoodsType','intDistance','fltRealWeight']]

C:\Users\user\AppData\Local\Temp\ipykernel_18896\1551313983.py:1: DtypeWarning: Columns (27,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df2=pd.read_csv('C:/Users/user/OneDrive - kyonggi.ac.kr/바탕 화면/화물맨프로젝트/화물맨 프로젝트/Ipynb/오토인코더추천시스템/tb_cargomylist_need_columns.csv', encoding='cp949')


In [162]:
selected_columns.iloc[37686]

strUpJAddress      경기 안성 보개면
strUploadType            지게차
strDownloadType          지게차
strDwJAddress      전남 곡성 오산면
strGoodsType            일반카고
intDistance              253
fltRealWeight           25.0
Name: 37686, dtype: object

In [163]:
selected_columns.iloc[13680]

strUpJAddress      경기 안산 단원구 성곡동
strUploadType                 일반
strDownloadType               일반
strDwJAddress      전남 고흥 나로호우주센터
strGoodsType                일반카고
intDistance                  371
fltRealWeight                5.5
Name: 13680, dtype: object

In [164]:
selected_columns.iloc[13712]

strUpJAddress      경기 이천 설성면
strUploadType             일반
strDownloadType           일반
strDwJAddress      서울 성동 성수동
strGoodsType            일반카고
intDistance               77
fltRealWeight           22.0
Name: 13712, dtype: object

In [165]:
selected_columns.iloc[37543]

strUpJAddress      경기 양주 삼숭동
strUploadType             일반
strDownloadType           일반
strDwJAddress          광주 광산
strGoodsType            일반카고
intDistance              338
fltRealWeight            7.0
Name: 37543, dtype: object

In [166]:
selected_columns.iloc[29748]

strUpJAddress      경기 평택 서탄면
strUploadType             일반
strDownloadType           일반
strDwJAddress      경기 시흥 과림동
strGoodsType            일반카고
intDistance               58
fltRealWeight           27.0
Name: 29748, dtype: object

In [167]:
value = 55238
user_info = user_selected_columns[user_selected_columns['intOwMemberSeq'] == value]
user_info

,strUpJAddress,intOwMemberSeq,strPaytype,strUploadType,strCarType,intDistance,strUpAreaCode,intMemberPrice,strCarSize
3,인천,55238.0,인수증,일반,카고,408,D000000,450000,25톤
15,인천,55238.0,인수증,일반,카고,408,D000000,450000,25톤
27,경기 수원 권선구 곡반정동,55238.0,인수증,지게차,카고,30,H130101,40000,1톤
29,경기 하남 망월동,55238.0,인수증,지게차,카고,89,H300007,120000,5톤
31,경기 이천 안흥동,55238.0,인수증,지게차,카고,177,H260017,130000,3 5톤
...,...,...,...,...,...,...,...,...,...
683948,충북 청주 흥덕구 옥산면,55238.0,인수증,일반,윙바디,123,J110201,130000,5톤
683952,충북 청주 흥덕구 옥산면,55238.0,인수증,일반,카고,139,J110201,110000,3 5톤
683953,충북 청주 흥덕구 옥산면,55238.0,인수증,일반,윙바디,139,J110201,170000,11톤
683955,충북 청주 흥덕구 옥산면,55238.0,인수증,일반,윙바디,47,J110201,70000,5톤
